In [13]:
import pandas as pd
df= pd.read_csv(r"C:\Users\Access\Documents\AI ROBOT\XAUUSD_Candlestick_1_M_BID_2020-1-1_2024-12-31.csv")

In [14]:
df=df.drop(columns='Volume')

In [15]:
import pandas as pd

# تحويل العمود الزمني إلى نوع datetime وتعيينه كمؤشر
df['Gmt time'] = pd.to_datetime(df['Gmt time'])
df.set_index('Gmt time', inplace=True)

# تحويل البيانات من 1 دقيقة إلى 5 دقائق
df_5m = df.resample('5T').agg({
    'Open': 'first',   # أول سعر افتتاح في الفاصل الزمني
    'High': 'max',     # أعلى سعر في الفاصل الزمني
    'Low': 'min',      # أقل سعر في الفاصل الزمني
    'Close': 'last'    # آخر سعر إغلاق في الفاصل الزمني
})

# عرض النتيجة
print(df_5m)

C:\Users\Access\AppData\Local\Temp\ipykernel_2976\422809626.py:8: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_5m = df.resample('5T').agg({


                         Open      High       Low     Close
Gmt time                                                   
2020-01-01 00:00:00  1516.915  1516.915  1516.915  1516.915
2020-01-01 00:05:00  1516.915  1516.915  1516.915  1516.915
2020-01-01 00:10:00  1516.915  1516.915  1516.915  1516.915
2020-01-01 00:15:00  1516.915  1516.915  1516.915  1516.915
2020-01-01 00:20:00  1516.915  1516.915  1516.915  1516.915
...                       ...       ...       ...       ...
2024-12-31 23:35:00  2624.255  2624.255  2624.255  2624.255
2024-12-31 23:40:00  2624.255  2624.255  2624.255  2624.255
2024-12-31 23:45:00  2624.255  2624.255  2624.255  2624.255
2024-12-31 23:50:00  2624.255  2624.255  2624.255  2624.255
2024-12-31 23:55:00  2624.255  2624.255  2624.255  2624.255

[526176 rows x 4 columns]


In [16]:
df_5m=df_5m.reset_index(drop=False) 

In [17]:
df_5m

,Gmt time,Open,High,Low,Close
0,2020-01-01 00:00:00,1516.915,1516.915,1516.915,1516.915
1,2020-01-01 00:05:00,1516.915,1516.915,1516.915,1516.915
2,2020-01-01 00:10:00,1516.915,1516.915,1516.915,1516.915
3,2020-01-01 00:15:00,1516.915,1516.915,1516.915,1516.915
4,2020-01-01 00:20:00,1516.915,1516.915,1516.915,1516.915
...,...,...,...,...,...
526171,2024-12-31 23:35:00,2624.255,2624.255,2624.255,2624.255
526172,2024-12-31 23:40:00,2624.255,2624.255,2624.255,2624.255
526173,2024-12-31 23:45:00,2624.255,2624.255,2624.255,2624.255
526174,2024-12-31 23:50:00,2624.255,2624.255,2624.255,2624.255


In [18]:
import pandas as pd

df = df_5m.copy()

# تعريف دالة التحقق من الشمعة الكبيرة (تبقى كما هي)
def is_large_body(candle):
    body = abs(candle['Open'] - candle['Close'])
    lower_shadow = abs(candle['Low'] - min(candle['Open'], candle['Close']))
    upper_shadow = abs(candle['High'] - max(candle['Open'], candle['Close']))
    return body > (lower_shadow + upper_shadow)

# تعديل دالة الهامر لتصبح شمعة Shooting Star (مقلوبة)
def is_shooting_star(candle):
    body = abs(candle['Open'] - candle['Close'])
    upper_shadow = abs(candle['High'] - max(candle['Open'], candle['Close']))
    lower_shadow = abs(candle['Low'] - min(candle['Open'], candle['Close']))
    return upper_shadow > 2 * body and lower_shadow < body

# تعديل دالة الصفقة لصفقات البيع
def calculate_sell_trade_shooting_star(row, df, tp_multiplier=1.5, sl_multiplier=1):
    idx = row.name
    if idx == df.index[0]:
        return 0, None, None
    
    first_candle = df.loc[df.index[df.index.get_loc(idx) - 1]]
    second_candle = row
    
    # الشرط المعكوس: شمعة صاعدة كبيرة تليها شمعة Shooting Star
    if is_large_body(first_candle) and first_candle['Close'] > first_candle['Open'] and is_shooting_star(second_candle):
        points = abs(first_candle['High'] - first_candle['Close'])
        take_profit = second_candle['Close'] - (points * tp_multiplier)
        stop_loss = second_candle['High'] + (points * sl_multiplier)
        return 1, take_profit, stop_loss
    
    return 0, None, None

# تطبيق الأعمدة الجديدة للصفقات البيعية
df['sell_trade_shooting'], df['tp_sell_shooting'], df['sl_sell_shooting'] = zip(*df.apply(
    lambda row: calculate_sell_trade_shooting_star(row, df), axis=1))

# تحديد نوع الصفقة للبيع
df['trade_type_shooting'] = df.apply(lambda row: -1 if row['sell_trade_shooting'] == 1 else 0, axis=1)

# دمج أعمدة TP و SL
df['tp_shooting'] = df.apply(lambda row: row['tp_sell_shooting'] if row['trade_type_shooting'] == -1 else None, axis=1)
df['sl_shooting'] = df.apply(lambda row: row['sl_sell_shooting'] if row['trade_type_shooting'] == -1 else None, axis=1)

# تعديل دالة تحديد نتيجة الصفقة للبيع
def determine_trade_result_shooting(row, df):
    idx = row.name
    if row['trade_type_shooting'] == -1:  # صفقة بيع
        for i in range(df.index.get_loc(idx) + 1, len(df)):
            future_row = df.iloc[i]
            if future_row['Low'] <= row['tp_shooting']:
                return 2  # رابحة
            elif future_row['High'] >= row['sl_shooting']:
                return 1  # خاسرة
    return 0  # لا توجد نتيجة

# تطبيق دالة النتائج
df['trade_result_shooting'] = df.apply(lambda row: determine_trade_result_shooting(row, df), axis=1)

# عرض النتائج
print(df[['Gmt time','Open', 'High', 'Low', 'Close', 'trade_type_shooting', 'trade_result_shooting', 'tp_shooting', 'sl_shooting']])

                  Gmt time      Open      High       Low     Close  \
0      2020-01-01 00:00:00  1516.915  1516.915  1516.915  1516.915   
1      2020-01-01 00:05:00  1516.915  1516.915  1516.915  1516.915   
2      2020-01-01 00:10:00  1516.915  1516.915  1516.915  1516.915   
3      2020-01-01 00:15:00  1516.915  1516.915  1516.915  1516.915   
4      2020-01-01 00:20:00  1516.915  1516.915  1516.915  1516.915   
...                    ...       ...       ...       ...       ...   
526171 2024-12-31 23:35:00  2624.255  2624.255  2624.255  2624.255   
526172 2024-12-31 23:40:00  2624.255  2624.255  2624.255  2624.255   
526173 2024-12-31 23:45:00  2624.255  2624.255  2624.255  2624.255   
526174 2024-12-31 23:50:00  2624.255  2624.255  2624.255  2624.255   
526175 2024-12-31 23:55:00  2624.255  2624.255  2624.255  2624.255   

        trade_type_shooting  trade_result_shooting  tp_shooting  sl_shooting  
0                         0                      0          NaN          NaN  
1

In [78]:
# حساب التكرارات والنسب المئوية
result = df['trade_result_shooting'].value_counts().sort_index()
percent = df['trade_result_shooting'].value_counts(normalize=True).sort_index() * 100

# دمج النتائج في DataFrame
result_df = pd.DataFrame({
    'Count': result,
    'Percentage (%)': percent.round(1)})

result_df['Percentage (%)'] = result_df['Percentage (%)'].astype(str) + '%'

print(result_df)

                       Count Percentage (%)
trade_result_shooting                      
1                        743          19.6%
2                       3052          80.4%


In [20]:
df_ = df[['Gmt time','Open', 'High', 'Low', 'Close', 'trade_type_shooting', 'trade_result_shooting', 'tp_shooting', 'sl_shooting']]

In [84]:
df_

,Gmt time,Open,High,Low,Close,trade_type_shooting,trade_result_shooting,tp_shooting,sl_shooting
0,2020-01-01 00:00:00,1516.915,1516.915,1516.915,1516.915,0,0,NaN,NaN
1,2020-01-01 00:05:00,1516.915,1516.915,1516.915,1516.915,0,0,NaN,NaN
2,2020-01-01 00:10:00,1516.915,1516.915,1516.915,1516.915,0,0,NaN,NaN
3,2020-01-01 00:15:00,1516.915,1516.915,1516.915,1516.915,0,0,NaN,NaN
4,2020-01-01 00:20:00,1516.915,1516.915,1516.915,1516.915,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
526171,2024-12-31 23:35:00,2624.255,2624.255,2624.255,2624.255,0,0,NaN,NaN
526172,2024-12-31 23:40:00,2624.255,2624.255,2624.255,2624.255,0,0,NaN,NaN
526173,2024-12-31 23:45:00,2624.255,2624.255,2624.255,2624.255,0,0,NaN,NaN
526174,2024-12-31 23:50:00,2624.255,2624.255,2624.255,2624.255,0,0,NaN,NaN


In [85]:
df_.to_csv(r"C:\Users\Access\Documents\AI ROBOT\XAUUSD_5m_trade_result_shooting_trade_list.csv", index=True)

In [55]:
import pandas as pd

df = df_.copy()

# تصفية الصفقات التي تحتوي على نتائج
df = df[df['trade_result_shooting'] != 0]

# استخراج التاريخ فقط
df['Gmt date'] = df['Gmt time'].dt.date

# تصنيف الصفقات
df['trade_type'] = df['trade_result_shooting'].apply(lambda x: 'رابحة' if x == 2 else 'خاسرة')

# حساب عدد الصفقات لكل يوم
daily_trades = df.groupby(['Gmt date', 'trade_type']).size().unstack(fill_value=0)

# إعادة تسمية الأعمدة
daily_trades = daily_trades.rename(columns={'رابحة': 'Winning Trades', 'خاسرة': 'Losing Trades'})

# إنشاء نطاق تواريخ شامل من البيانات الأصلية
start_date = df_['Gmt time'].min().normalize()
end_date = df_['Gmt time'].max().normalize()
all_dates = pd.date_range(start=start_date, end=end_date, freq='D')

# تحويل الفهرس إلى تنسيق تاريخي
daily_trades.index = pd.to_datetime(daily_trades.index)

# إعادة الفهرس ليشمل جميع التواريخ
daily_trades = daily_trades.reindex(all_dates, fill_value=0)

# تصفية أيام السبت (5) والأحد (6) باستخدام weekday
daily_trades = daily_trades[daily_trades.index.weekday < 5]  # 0-4 = من الاثنين إلى الجمعة

# إضافة عمود الصفقات الكلية
daily_trades['Total Trades'] = daily_trades['Winning Trades'] + daily_trades['Losing Trades']

# إعادة تنسيق الفهرس لعرضه كتواريخ بدون وقت
daily_trades.index = daily_trades.index.date

print(daily_trades)

trade_type  Losing Trades  Winning Trades  Total Trades
2020-01-01              0               0             0
2020-01-02              0               3             3
2020-01-03              0               2             2
2020-01-06              0               1             1
2020-01-07              0               6             6
...                   ...             ...           ...
2024-12-25              0               0             0
2024-12-26              0               1             1
2024-12-27              0               0             0
2024-12-30              0               4             4
2024-12-31              1               6             7

[1305 rows x 3 columns]


In [81]:
daily_trades[:60]

trade_type,Losing Trades,Winning Trades,Total Trades
2020-01-01,0,0,0
2020-01-02,0,3,3
2020-01-03,0,2,2
2020-01-06,0,1,1
2020-01-07,0,6,6
2020-01-08,0,2,2
2020-01-09,0,1,1
2020-01-10,2,3,5
2020-01-13,0,3,3
2020-01-14,0,3,3


In [83]:
daily_trades.to_csv(r"C:\Users\Access\Documents\AI ROBOT\XAUUSD_5m_trade_result_shooting.csv", index=True)

In [57]:
daily_trades.describe().round(1).drop(['count','std'])

trade_type,Losing Trades,Winning Trades,Total Trades
mean,0.6,2.3,2.9
min,0.0,0.0,0.0
25%,0.0,1.0,2.0
50%,0.0,2.0,3.0
75%,1.0,3.0,4.0
max,5.0,8.0,9.0


In [80]:
# حساب التكرارات والنسب المئوية
total_counts = daily_trades['Total Trades'].value_counts()
total_percent = daily_trades['Total Trades'].value_counts(normalize=True) * 100

# دمج النتائج في DataFrame
result_df = pd.DataFrame({
    'Count': total_counts,
    'Percentage (%)': total_percent.round(1)
})

result_df['Percentage (%)'] = result_df['Percentage (%)'].astype(str) + '%'

# طباعة النتيجة
print(result_df)

              Count Percentage (%)
Total Trades                      
2               273          20.9%
3               249          19.1%
1               231          17.7%
4               223          17.1%
5               128           9.8%
0                95           7.3%
6                58           4.4%
7                32           2.5%
8                15           1.1%
9                 1           0.1%


In [76]:
# حساب التكرارات والنسب المئوية
winning_counts = daily_trades['Winning Trades'].value_counts()
winning_percent = daily_trades['Winning Trades'].value_counts(normalize=True) * 100

# دمج النتائج في DataFrame
result_df = pd.DataFrame({
    'Count': winning_counts,
    'Percentage (%)': winning_percent.round(1)
})

result_df['Percentage (%)'] = result_df['Percentage (%)'].astype(str) + '%'

# طباعة النتيجة
print(result_df)

                Count Percentage (%)
Winning Trades                      
1                 300          23.0%
2                 297          22.8%
3                 263          20.2%
4                 174          13.3%
0                 155          11.9%
5                  70           5.4%
6                  31           2.4%
7                  13           1.0%
8                   2           0.2%


In [79]:
# حساب التكرارات والنسب المئوية
losing_counts = daily_trades['Losing Trades'].value_counts().sort_index()
losing_percent = daily_trades['Losing Trades'].value_counts(normalize=True).sort_index() * 100

# دمج النتائج في DataFrame
result_df = pd.DataFrame({
    'Count': losing_counts,
    'Percentage (%)': losing_percent.round(1)
})

result_df['Percentage (%)'] = result_df['Percentage (%)'].astype(str) + '%'

# طباعة النتيجة
print(result_df)

               Count Percentage (%)
Losing Trades                      
0                764          58.5%
1                389          29.8%
2                117           9.0%
3                 29           2.2%
4                  5           0.4%
5                  1           0.1%
